# Compute Berezenian of sadle manifold
$$\displaystyle Q=U_{\eta }^{-1} Q_{0} U_{\eta } ,\ Q_{0} \ =\ \Lambda e^{W}, \; \{W,\Lambda\}=0, [U_\eta, \Lambda]=0$$

\begin{equation*}
Q_{0}{}_{FF} =\begin{pmatrix}
cos\theta _{F} & ie^{i\phi _{F}}\sin \theta _{F}\\
-ie^{-i\phi _{F}}\sin \theta _{F} & -cos\theta _{F}
\end{pmatrix} ,\ Q_{0}{}_{BB} =\begin{pmatrix}
\cosh \ \theta _{B} & e^{i\phi _{B}}\sinh \theta _{B}\\
-e^{-i\phi _{B}}\sinh \theta _{B} & -\cosh \ \theta _{B}
\end{pmatrix}
\end{equation*}

\begin{equation*}
U_{\eta }^{RR} =\begin{pmatrix}
1\ +\frac{\eta _{FB}^{RR} \eta _{BF}^{RR}}{2} & \eta _{FB}^{RR}\\
-\eta _{BF}^{RR} & 1\ +\frac{\eta _{BF}^{RR} \eta _{FB}^{RR}}{2}
\end{pmatrix} \Longrightarrow Q\ =\ \begin{pmatrix}
U{_{\eta }^{-1}}^{RR} Q{_{0}}^{RR} U{_{\eta }}^{RR} & U{_{\eta }^{-1}}^{RR} Q{_{0}}^{RA} U{_{\eta }}^{AR}\\
U{_{\eta }^{-1}}^{AA} Q{_{0}}^{AR} U{_{\eta }}^{RR} & U{_{\eta }^{-1}}^{AA} Q{_{0}}^{AA} U{_{\eta }}^{AA}
\end{pmatrix}
\end{equation*}

Length square is
\begin{equation*}
dl^2 = \text{str} \ \text{(d} Q)^{2} \ =\ \text{str} \ \text{(d} Q^{RR})^{2} \ +\ \text{(d} Q^{AA})^{2} \ +\ \text{d} Q^{AR}\text{d} Q^{RA} +\ \text{d} Q^{RA}\text{d} Q^{AR}
\end{equation*}

Quadratic form
$$
dl^2 = d\Phi_i M_{ij} d\Phi_j
$$
with supermatrix $M$, berezenian is $1/\sqrt{sdet M}$

In [1]:
# Make grassman algebra


from sympy import Symbol, S

class AnticomSym(Symbol):
    def __new__(cls,*args,**kwargs):
        return super().__new__(cls,*args,**kwargs,commutative=False)

    def __mul__(self,other):
        if isinstance(other,AnticomSym):
            if other==self:
                return S.Zero
            elif other.name<self.name:
                return -Symbol.__mul__(other,self)

        return super().__mul__(other)

    def __pow__(self,exponent):
        if exponent>=2:
            return S.Zero
        else:
            return super().__pow__(exponent)
        
def sort_product(product): # simplify complex prodact like x*y*x = - x*x*y = 0 
    while True:
        if not isinstance(product,Mul):
            return product

        arglist = list(product.args)
        i = 0
        while i < len(arglist)-1:
            slice_prod = arglist[i]*arglist[i+1]
            is_mul = isinstance(slice_prod,Mul)
            arglist[i:i+2] = slice_prod.args if is_mul else [slice_prod]
            i += 1

        new_product = Mul(*arglist)
        if product == new_product:
            return new_product
        product = new_product
        
        
def sort_products(expr): # reccurent simplify complex expresion
    if expr.is_Atom:
        return expr
    else:
        simplified_args = (sort_products(arg) for arg in expr.args)
        if isinstance(expr,Mul):
            return sort_product(Mul(*simplified_args))
        else:
            return expr.func(*simplified_args)

In [2]:
from sympy import *

In [3]:
# Define grassman vars with diff

eta_RRFB = AnticomSym("eta_RRFB")
deta_RRFB = AnticomSym("d\\eta_RRFB")

eta_RRBF = AnticomSym("eta_RRBF")
deta_RRBF = AnticomSym("d\\eta_RRBF")

eta_AAFB = AnticomSym("eta_AAFB")
deta_AAFB = AnticomSym("d\\eta_AAFB")

eta_AABF = AnticomSym("eta_AABF")
deta_AABF = AnticomSym("d\\eta_AABF")

anticomute_var = {'eta_RRFB': (eta_RRFB, deta_RRFB),'eta_RRBF': (eta_RRBF, deta_RRBF),
                  'eta_AAFB': (eta_AAFB, deta_AAFB),'eta_AABF': (eta_AABF, deta_AABF)} 

In [5]:
# Define grasman generator
W_2_RRFB = Matrix([[0, eta_RRFB],[eta_RRBF, 0]])
W_2_RRFB

Matrix([
[       0, eta_RRFB],
[eta_RRBF,        0]])

In [6]:
# End group element
U_eta_RR =eye(2)+ W_2_RRFB + W_2_RRFB**2/2
U_eta_RR

Matrix([
[1 - eta_RRBF*eta_RRFB/2,                eta_RRFB],
[               eta_RRBF, 1 + eta_RRBF*eta_RRFB/2]])

In [7]:
U_eta_RR_inv = eye(2) -  W_2_RRFB + W_2_RRFB**2/2
U_eta_RR_inv

Matrix([
[1 - eta_RRBF*eta_RRFB/2,               -eta_RRFB],
[              -eta_RRBF, 1 + eta_RRBF*eta_RRFB/2]])

In [8]:
# Def commute vars
theta_F, phi_F = symbols('theta_F, phi_F', real=True)
theta_B, phi_B = symbols('theta_B, phi_B', real=True)

dtheta_F, dphi_F = symbols(r'd\theta_F, d\phi_F', real=True)
dtheta_B, dphi_B = symbols(r'd\theta_B, d\phi_B', real=True)

comute_var = {'theta_F': (theta_F, dtheta_F), 'theta_B': (theta_B, dtheta_B), 
              'phi_F':(phi_F, dphi_F), 'phi_B':(phi_B, dphi_B)} 

In [10]:
# Def Q_0
Q_0_RR = Matrix([[cos(theta_F),0],[0, cosh(theta_B)]])
Q_0_RR

In [13]:
# Find Q_RR

Q_RR = sort_products(expand(U_eta_RR_inv @ Q_0_RR @ U_eta_RR))
Q_RR

Matrix([
[cos(theta_F) - cos(theta_F)*eta_RRBF*eta_RRFB + cosh(theta_B)*eta_RRBF*eta_RRFB,                                    cos(theta_F)*eta_RRFB - cosh(theta_B)*eta_RRFB],
[                                -cos(theta_F)*eta_RRBF + cosh(theta_B)*eta_RRBF, -cos(theta_F)*eta_RRBF*eta_RRFB + cosh(theta_B) + cosh(theta_B)*eta_RRBF*eta_RRFB]])

In [14]:
# same steps for AA
W_2_AAFB = Matrix([[0, eta_AAFB],[eta_AABF, 0]])
W_2_AAFB

Matrix([
[       0, eta_AAFB],
[eta_AABF,        0]])

In [15]:
U_eta_AA =eye(2)+ W_2_AAFB + W_2_AAFB**2/2
U_eta_AA

Matrix([
[1 - eta_AABF*eta_AAFB/2,                eta_AAFB],
[               eta_AABF, 1 + eta_AABF*eta_AAFB/2]])

In [16]:
U_eta_AA_inv = eye(2) -  W_2_AAFB + W_2_AAFB**2/2
U_eta_AA_inv

Matrix([
[1 - eta_AABF*eta_AAFB/2,               -eta_AAFB],
[              -eta_AABF, 1 + eta_AABF*eta_AAFB/2]])

In [17]:
Q_0_AA = - Q_0_RR
Q_0_AA

In [19]:
Q_AA = sort_products(expand(U_eta_AA_inv @ Q_0_AA @ U_eta_AA))
Q_AA

Matrix([
[-cos(theta_F) + cos(theta_F)*eta_AABF*eta_AAFB - cosh(theta_B)*eta_AABF*eta_AAFB,                                  -cos(theta_F)*eta_AAFB + cosh(theta_B)*eta_AAFB],
[                                  cos(theta_F)*eta_AABF - cosh(theta_B)*eta_AABF, cos(theta_F)*eta_AABF*eta_AAFB - cosh(theta_B) - cosh(theta_B)*eta_AABF*eta_AAFB]])

In [20]:
## Make off diag Q
Q_0_RA = Matrix([[I*exp(I*phi_F)*sin(theta_F),0],[0, exp(I*phi_B)*sinh(theta_B)]]) 
Q_0_RA

Matrix([
[I*exp(I*phi_F)*sin(theta_F),                          0],
[                          0, exp(I*phi_B)*sinh(theta_B)]])

In [21]:
Q_RA = sort_products(expand(U_eta_RR_inv @ Q_0_RA @ U_eta_AA))
Q_RA

Matrix([
[exp(I*phi_B)*sinh(theta_B)*eta_AABF*eta_RRFB + I*exp(I*phi_F)*sin(theta_F) - I*exp(I*phi_F)*sin(theta_F)*eta_AABF*eta_AAFB/2 + I*exp(I*phi_F)*sin(theta_F)*eta_AABF*eta_AAFB*eta_RRBF*eta_RRFB/4 - I*exp(I*phi_F)*sin(theta_F)*eta_RRBF*eta_RRFB/2,                                                -exp(I*phi_B)*sinh(theta_B)*eta_AABF*eta_AAFB*eta_RRFB/2 - exp(I*phi_B)*sinh(theta_B)*eta_RRFB + I*exp(I*phi_F)*sin(theta_F)*eta_AAFB - I*exp(I*phi_F)*sin(theta_F)*eta_AAFB*eta_RRBF*eta_RRFB/2],
[                                                   exp(I*phi_B)*sinh(theta_B)*eta_AABF + exp(I*phi_B)*sinh(theta_B)*eta_AABF*eta_RRBF*eta_RRFB/2 + I*exp(I*phi_F)*sin(theta_F)*eta_AABF*eta_AAFB*eta_RRBF/2 - I*exp(I*phi_F)*sin(theta_F)*eta_RRBF, exp(I*phi_B)*sinh(theta_B) + exp(I*phi_B)*sinh(theta_B)*eta_AABF*eta_AAFB/2 + exp(I*phi_B)*sinh(theta_B)*eta_AABF*eta_AAFB*eta_RRBF*eta_RRFB/4 + exp(I*phi_B)*sinh(theta_B)*eta_RRBF*eta_RRFB/2 + I*exp(I*phi_F)*sin(theta_F)*eta_AAFB*eta_RRBF]])

In [22]:
Q_0_AR = -Matrix([[I*exp(-I*phi_F)*sin(theta_F),0],[0, exp(-I*phi_B)*sinh(theta_B)]]) 
Q_0_AR

Matrix([
[-I*exp(-I*phi_F)*sin(theta_F),                            0],
[                            0, -exp(-I*phi_B)*sinh(theta_B)]])

In [23]:
Q_AR = sort_products(expand(U_eta_AA_inv @ Q_0_AR @ U_eta_RR))
Q_AR

Matrix([
[-I*exp(-I*phi_F)*sin(theta_F) + I*exp(-I*phi_F)*sin(theta_F)*eta_AABF*eta_AAFB/2 - I*exp(-I*phi_F)*sin(theta_F)*eta_AABF*eta_AAFB*eta_RRBF*eta_RRFB/4 + I*exp(-I*phi_F)*sin(theta_F)*eta_RRBF*eta_RRFB/2 + exp(-I*phi_B)*sinh(theta_B)*eta_AAFB*eta_RRBF,                                                  I*exp(-I*phi_F)*sin(theta_F)*eta_AABF*eta_AAFB*eta_RRFB/2 - I*exp(-I*phi_F)*sin(theta_F)*eta_RRFB + exp(-I*phi_B)*sinh(theta_B)*eta_AAFB + exp(-I*phi_B)*sinh(theta_B)*eta_AAFB*eta_RRBF*eta_RRFB/2],
[                                                     I*exp(-I*phi_F)*sin(theta_F)*eta_AABF - I*exp(-I*phi_F)*sin(theta_F)*eta_AABF*eta_RRBF*eta_RRFB/2 - exp(-I*phi_B)*sinh(theta_B)*eta_AABF*eta_AAFB*eta_RRBF/2 - exp(-I*phi_B)*sinh(theta_B)*eta_RRBF, I*exp(-I*phi_F)*sin(theta_F)*eta_AABF*eta_RRFB - exp(-I*phi_B)*sinh(theta_B) - exp(-I*phi_B)*sinh(theta_B)*eta_AABF*eta_AAFB/2 - exp(-I*phi_B)*sinh(theta_B)*eta_AABF*eta_AAFB*eta_RRBF*eta_RRFB/4 - exp(-I*phi_B)*sinh(theta_B)*eta_RRBF*eta_RRFB

In [24]:
def get_d(expr):
    """
    Returns differential form of simple expr use var dict
    """
    res = expr*0
    for key in comute_var.keys():
        res += diff(expr, comute_var[key][0])*comute_var[key][1]
    for key in anticomute_var.keys():
        res += expand(expr - expr.replace(anticomute_var[key][0],0)).replace(anticomute_var[key][0],anticomute_var[key][1])
    return res

In [25]:
def get_d_matr(matr):
    """
    Returns differential form of 2x2 matrix
    """
    res = zeros(2,2)
    for i in range(2):
        for j in range(2):
            res[i,j] = get_d(matr[i,j])
    return res

In [26]:
# get dQ
dQ_RR = get_d_matr(Q_RR)
dQ_AA = get_d_matr(Q_AA)
dQ_RA = get_d_matr(Q_RA)
dQ_AR = get_d_matr(Q_AR)

In [27]:
res = (dQ_RR @ dQ_RR + dQ_AA @ dQ_AA + dQ_AR @ dQ_RA + dQ_RA @ dQ_AR) # expr under supertrace
dl_sqr = res[0,0] - res[1,1]
dl_sqr = sort_products(expand(dl_sqr))

In [60]:
# Make matrix of quadratic forms
M_2 = zeros(8,8)
i=0
for key_i in anticomute_var.keys():
    j=0
    for key_j in anticomute_var.keys(): ## FF block
        print(i,j)
        if i<j:
            expr=(dl_sqr - expand(dl_sqr.subs(anticomute_var[key_i][1],0)))
            expr = expr - expr.subs(anticomute_var[key_j][1],0)
            M_2[i,j] = -(factor(simplify(factor(expr)))/2).subs(anticomute_var[key_i][1],1).subs(anticomute_var[key_j][1],1)
            # - due to contradiction of order of dictionary and alphabetic names
            M_2[j,i] = -M_2[i,j] #symmetry
        j+=1
    for key_j in comute_var.keys(): ## FB & BF block
        print(i,j)
        if i<j:
            expr=(dl_sqr - expand(dl_sqr.subs(anticomute_var[key_i][1],0)))
            expr = expr - expr.subs(comute_var[key_j][1],0)
            M_2[i,j] = (factor(simplify(factor(expr)))/2).subs(anticomute_var[key_i][1],1).subs(comute_var[key_j][1],1)
            M_2[j,i] = - M_2[i,j] #symmetry
        j+=1
    i+=1

for key_i in comute_var.keys(): ##
    j=0
    for key_j in anticomute_var.keys():
        print(i,j)
        j+=1
    for key_j in comute_var.keys(): ## BB block
        print(i,j)
        if i<j:
            expr=(dl_sqr - expand(dl_sqr.subs(comute_var[key_i][1],0)))
            expr = expr - expr.subs(comute_var[key_j][1],0)
            M_2[i,j] = factor(simplify(factor(expr)))/2
            M_2[j,i] = factor(simplify(factor(expr)))/2
        j+=1
    i+=1 
    
    
i = 4 
for key_i in comute_var.keys(): ## diagonal of BB block
    M_2[i,i] = diff(dl_sqr, comute_var[key_i][1], comute_var[key_i][1])/2
    i+=1

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7


In [61]:
M_2 = simplify(M_2)

In [62]:
M_2_FF = M_2[:4,:4]
M_2_FF

Matrix([
[                                                                                                                                   0,                        (sin(theta_F)**2*eta_RRBF*eta_RRFB + 4*cos(theta_F)*cosh(theta_B) + sinh(theta_B)**2*eta_RRBF*eta_RRFB - 4)/2,                                                                         -(cos(theta_F)**2/2 - cosh(theta_B)**2/2)*eta_AABF*eta_RRBF, (-4*I*exp(I*(phi_B - phi_F))*sin(theta_F)*sinh(theta_B) + sin(theta_F)**2*eta_AAFB*eta_RRBF + sinh(theta_B)**2*eta_AAFB*eta_RRBF)/2],
[                      -(sin(theta_F)**2*eta_RRBF*eta_RRFB + 4*cos(theta_F)*cosh(theta_B) + sinh(theta_B)**2*eta_RRBF*eta_RRFB - 4)/2,                                                                                                                                    0, (sin(theta_F)**2*eta_AABF*eta_RRFB + sinh(theta_B)**2*eta_AABF*eta_RRFB + 4*I*exp(-I*(phi_B - phi_F))*sin(theta_F)*sinh(theta_B))/2,                                                      

In [63]:
M_2_BB = M_2[4:,4:]
M_2_BB

Matrix([
[2, 0,                 0,                  0],
[0, 2,                 0,                  0],
[0, 0, 2*sin(theta_F)**2,                  0],
[0, 0,                 0, 2*sinh(theta_B)**2]])

In [64]:
M_2_FB = M_2[:4,4:]
M_2_FB

Matrix([
[0, 0, -I*sin(theta_F)**2*eta_RRBF, -I*sinh(theta_B)**2*eta_RRBF],
[0, 0, -I*sin(theta_F)**2*eta_RRFB, -I*sinh(theta_B)**2*eta_RRFB],
[0, 0,  I*sin(theta_F)**2*eta_AABF,  I*sinh(theta_B)**2*eta_AABF],
[0, 0,  I*sin(theta_F)**2*eta_AAFB,  I*sinh(theta_B)**2*eta_AAFB]])

In [65]:
M_2_BF = M_2[4:,:4]
M_2_BF

Matrix([
[                          0,                           0,                            0,                            0],
[                          0,                           0,                            0,                            0],
[ I*sin(theta_F)**2*eta_RRBF,  I*sin(theta_F)**2*eta_RRFB,  -I*sin(theta_F)**2*eta_AABF,  -I*sin(theta_F)**2*eta_AAFB],
[I*sinh(theta_B)**2*eta_RRBF, I*sinh(theta_B)**2*eta_RRFB, -I*sinh(theta_B)**2*eta_AABF, -I*sinh(theta_B)**2*eta_AAFB]])

$$
sdet M = \frac{det(M_{FF} - M_{FB}M_{BB}^{-1}M_{BF})}{det M_{BB}}
$$

In [66]:
ch_mat = (sort_products(expand((M_2_FF - M_2_FB @ M_2_BB.inv() @ M_2_BF)))) ## matrix in numerator
ch_mat = simplify(ch_mat)
ch_mat

In [71]:
factor(ch_mat.det())

16*(sin(theta_F)**2*sinh(theta_B)**2 + cos(theta_F)**2*cosh(theta_B)**2 - 2*cos(theta_F)*cosh(theta_B) + 1)**2

In [73]:
sqrt(M_2_BB.det()/factor(ch_mat.det()))

Abs(sin(theta_F))*Abs(sinh(theta_B))/Abs(sin(theta_F)**2*sinh(theta_B)**2 + cos(theta_F)**2*cosh(theta_B)**2 - 2*cos(theta_F)*cosh(theta_B) + 1)

In [80]:
simplify((sqrt(M_2_BB.det()/factor(ch_mat.det()))).replace(sinh(theta_B)**2, cosh(theta_B)**2-1).replace(sin(theta_F)**2, 1-cos(theta_F)**2))

Abs(sin(theta_F)*sinh(theta_B)/(cos(theta_F) - cosh(theta_B))**2)

In [82]:
print(latex(simplify((sqrt(M_2_BB.det()/factor(ch_mat.det()))).replace(sinh(theta_B)**2, 
                                                                       cosh(theta_B)**2-1).replace(sin(theta_F)**2, 1-cos(theta_F)**2))))

\left|{\frac{\sin{\left(\theta_{F} \right)} \sinh{\left(\theta_{B} \right)}}{\left(\cos{\left(\theta_{F} \right)} - \cosh{\left(\theta_{B} \right)}\right)^{2}}}\right|


Therefore $J$ is:

$$
J = \left|{\frac{\sin{\left(\theta_{F} \right)} \sinh{\left(\theta_{B} \right)}}{\left(\cos{\left(\theta_{F} \right)} - \cosh{\left(\theta_{B} \right)}\right)^{2}}}\right|
$$